In [ ]:
import sys

sys.argv = [""]
del sys

In [ ]:
import argparse
import os
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# causing the kernal to die.!!
from inpaint.data import *
# from inpaint.data import PlacesDataset 
# from inpaint.utils import random_bbox_mask, random_ff_mask
# from inpaint.utils.metrics import psnr
# from inpaint.utils.metrics import ssim

import matplotlib.pyplot as plt

import random

In [ ]:
def flipCoin():
    f = random.random()
    return True if f < 0.5 else False

In [ ]:
def create_cfg(args):
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)

    cfg = parser.parse_args()
    return cfg

In [ ]:
args = {}
# model
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = False

# dataset
args["--val_ds_dir"] = '../../samples/Places365'
args["--crop_size"] = (256, 256)
args["--batch_size"] =  1
args["--num_workers"] = 1
args["--device_id"] = 0
args["--val_steps"] = 10

In [ ]:
cfg = create_cfg(args)

In [ ]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=cfg.crop_size)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

In [ ]:
print(f"Total validation images: {len(val_ds)}")

___

In [ ]:
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Evaluate

___

1. Initialize generator from a saved checkpoint

In [ ]:
# load checkpoint and initialize
checkpointslist = os.listdir(checkpoints_dir)

for i in range(len(checkpointslist)):
    checkpointslist[i] = checkpoints_dir + "/" + checkpointslist[i]
    print(checkpointslist)

generator = GatedGenerator(self.cfg)
# generator = None

___

2. Create Evaluate instance pass dataloader and generator instance to Evaluate 

In [ ]:
eval = Evaluate(cfg, generator, val_loader)

____
3. Evaluate (return avg psnr and avg ssim value

In [ ]:
avg_psnr, avg_ssim = eval.evaluate()